In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

import torch
import torch_geometric
from glob import glob
from tqdm.auto import tqdm, trange
import numpy as np
import matplotlib.pyplot as plt

import pytorch_lightning as pl
torch.set_printoptions(sci_mode=False)
# device = torch.device('cuda')
device = torch.device('cpu')
# pl.seed_everything(12345)

In [30]:
# td_data = load_atis_data(filename, flipX=0, flipY=0)
#
# Loads data from files generated by the ATIS CAMERA
# This function only read (t,x,y,p) and discard other fields (if
# any) of events.
# timestamps are in uS
# td_data is a structure containing the fields ts, x, y and p
#
# flipX, flipY allow to flip the image arround the X and Y axes. If these values
# are non zero, the corresponding dimension will be flipped considering its size
# to be the value contained in the 'flip' variable (i.e. X = flipX - X)
# (They defaults to 0 if non-specified)

def load_atis_data(filename: str, flipX: int = 0, flipY: int = 0):
    from math import floor
    import struct
    import numpy as np

    f = open(filename, 'rb')

    # Parse header if any
    header = []
    endOfHeader = 0
    numCommentLine = 0
    while (endOfHeader == 0):
        bod = f.tell()
        tline = f.readline(256)

        if (tline[0] != ord('%')):
            endOfHeader = 1
        else:
            tline_str = tline.decode('ascii')
            words = tline_str.split()
            if (len(words) > 2 ):
                if (words[1] == 'Date'):
                    if (len(words) > 3):
                        date = words[2] + ' ' + words[3]
                        header.append([words[1], date])
                else:
                    header.append([words[1], words[2]])


            numCommentLine = numCommentLine + 1


    f.seek(bod, 0) # seek from the start of the file

    evType = 0
    evSize = 8
    if (numCommentLine > 0): # Ensure compatibility with previous files.
        # Read event type
        evType = struct.unpack('B', f.read(1))[0]
        # Read event size
        evSize = struct.unpack('B', f.read(1))[0]


    bof = f.tell() # from the start of event data
    f.seek(0,2) # move to the end of the file
    numEvents = floor((f.tell()-bof)/evSize)
    f.close()

    # read data
    data = np.fromfile(filename, dtype=np.int32, offset=bof).reshape((-1,2))
    allTs = data[:, 0]
    allAddr = data[:, 1]


    # td_data.ts = double(allTs)

    # version = 0
    # index = find(strcmp(header(:,1), 'Version'))
    # if (~isempty(index)):
    #     version = header{index, 2}


    # xmask = hex2dec('00003FFF')
    # ymask = hex2dec('0FFFC000')
    # polmask = hex2dec('10000000')
    # xshift=0 # bits to shift x to right
    # yshift=14 # bits to shift y to right
    # polshift=28 # bits to shift p to right

    # addr=abs(allAddr) # make sure nonnegative or an error will result from bitand (glitches can somehow result in negative addressses...)
    # td_data.x=double(bitshift(bitand(addr,xmask),-xshift)) # x addresses
    # td_data.y=double(bitshift(bitand(addr,ymask),-yshift)) # y addresses
    # # td_data.p=-1+2*double(bitshift(bitand(addr,polmask),-polshift)); # 1 for ON, -1 for OFF
    # td_data.p=double(bitshift(bitand(addr,polmask),-polshift)) # 1 for ON, -1 for OFF

    # if (flipX > 0):
    #     td_data.x = flipX - td_data.x

    # if (flipY > 0):
    #     td_data.y = flipY - td_data.y

    return td_data


filename = '/users/yyang22/thesis/aegnn_project/obj_004396_td.dat'
td_data = load_atis_data(filename, flipX=0, flipY=0)
print(td_data)


b'\x00\x00\x00\x00'
